In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from datasets import load_dataset
import evaluate as evaluate
from transformers import get_scheduler
from transformers import AutoModelForSequenceClassification
import argparse
import subprocess

/Users/wangjiarui/miniconda3/envs/ml/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
model_name = "distilbert-base-uncased"
dataset = load_dataset("boolq")
dataset = dataset.shuffle()  # shuffle the data

dataset_train_subset = dataset['train'][:10]
dataset_dev_subset = dataset['train'][:10]
dataset_test_subset = dataset['train'][:10]
mytokenizer = AutoTokenizer.from_pretrained(model_name)

Found cached dataset boolq (/Users/wangjiarui/.cache/huggingface/datasets/boolq/default/0.1.0/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
class BoolQADataset(torch.utils.data.Dataset):
    """
    Dataset for the dataset of BoolQ questions and answers
    """

    def __init__(self, passages, questions, answers, tokenizer, max_len):
        self.passages = passages
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.answers)

    def __getitem__(self, index):
        """
        This function is called by the DataLoader to get an instance of the data
        :param index:
        :return:
        """

        passage = str(self.passages[index])
        question = self.questions[index]
        answer = self.answers[index]

        # this is input encoding for your model. Note, question comes first since we are doing question answering
        # and we don't wnt it to be truncated if the passage is too long
        input_encoding = question + " [SEP] " + passage

        # encode_plus will encode the input and return a dictionary of tensors
        encoded_review = self.tokenizer.encode_plus(
            input_encoding,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors="pt",
            padding="max_length",
            truncation=True
        )

        return {
            'input_ids': encoded_review['input_ids'][0],  # we only have one example in the batch
            'attention_mask': encoded_review['attention_mask'][0],
            # attention mask tells the model where tokens are padding
            'labels': torch.tensor(answer, dtype=torch.long)  # labels are the answers (yes/no)
        }


In [14]:
train_dataset = BoolQADataset(
        passages=list(dataset_train_subset['passage']),
        questions=list(dataset_train_subset['question']),
        answers=list(dataset_train_subset['answer']),
        tokenizer=mytokenizer,
        max_len=128
    )
train_dataloader = DataLoader(train_dataset, batch_size=2)

In [16]:
for batch in train_dataloader:
    input_ids = batch['input_ids']
    mask = batch['attention_mask']
    labels = batch['labels']



torch.Size([2, 128]) <class 'torch.Tensor'>
torch.Size([2, 128]) <class 'torch.Tensor'>

torch.Size([2, 128]) <class 'torch.Tensor'>
torch.Size([2, 128]) <class 'torch.Tensor'>

torch.Size([2, 128]) <class 'torch.Tensor'>
torch.Size([2, 128]) <class 'torch.Tensor'>

torch.Size([2, 128]) <class 'torch.Tensor'>
torch.Size([2, 128]) <class 'torch.Tensor'>

torch.Size([2, 128]) <class 'torch.Tensor'>
torch.Size([2, 128]) <class 'torch.Tensor'>

